# How Does PCA Work? 

## Data Cleaning and Scaling

Before we look at the algorithm, we will read in the abalone data [discussed previously](./setup/datasets.html) and *clean* it for processing. 

In [ ]:
import pandas as pd

data = pd.read_csv('./../data/abalone.csv')
data

As mentioned, the `sex` data is categorical and, therefore, incompatible with the PCA algorithm in the form that we will discuss. 
So, we should remove that. 

In [ ]:
cleaned = data[data.columns[1:]]

The remaining features are continuous variables and can be used for the PCA algorithm. 
```{warning}
Here, we highlight that the `rings` data is not strictly continuous, as it can only be integer values.
We refer to data of this type as **discrete**. 
However, as it does not describe a *class* of the feature, like `sex`, it is compatible with PCA. 
```

If we look at the variances for each feature, we will see that some features (`rings`) cover much larger ranges than others (`diameter`). 
Therefore, if we naïvely applied the PCA algorithm, we risk the first principal component containing only really information about the `rings`, as these have the most variance. 

In [ ]:
cleaned.var()

Hence, we must scale our data. 
There are a few approaches to this, but here, we will scale our data such that the mean for each feature is 0 and the variance is 1. 
In `scikit-learn` this is called the [`StandardScaler()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler), but here we will do it manually to show how it is achieved. 

In [ ]:
scaled = (cleaned - cleaned.mean()) / cleaned.std()
scaled.var()

Our data is now ready for the PCA algorithm. 

## The PCA Algorithm

Strictly speaking, the PCA algorithm involves the calculation of the covariance matrix for the data, followed by determining the eigenvalues and eigenvectors.
The eigenvectors are the principal components, which can be sorted by their eigenvalues, which are the explained variance. 
We can see this in action for our data below. 
````{margin}
```{note}
The `np.argsort` function returns the indices of the input sorted numerically increasing. 
Therefore, we flip these indices as we want the largest eigenvalue first. 
```
````

In [ ]:
import numpy as np

cov = scaled.cov()
eigenthings = np.linalg.eig(cov)
indices = np.argsort(eigenthings.eigenvalues)[::-1]
explained_variance = eigenthings.eigenvalues[indices]
components = eigenthings.eigenvectors[:, indices].T

Let's look at the covariance matrix from our input data after the linear transformation by the matrix that our PCA algorithm defines.

In [ ]:
transformed = components / np.sqrt(explained_variance[:, np.newaxis]) @ scaled.T
scaled_with_pc = scaled.copy()
for i in range(8):
    scaled_with_pc[f'PC{i+1}'] = transformed.loc[i]
scaled_with_pc[[f'PC{i+1}' for i in range(cleaned.shape[1])]].cov()

We can see that this is a diagonal matrix, as we would expect. 
````{margin}
```{note}
The SVD approach to PCA is more flexible by allowing "truncated PCA", where not all components are computed, but just the top *n*. 
```
````

However, calculating the covariance matrix can be slow, where there are a large number of data points. 
Therefore, a more stable (and flexible) approach is to use singular value decomposition (this is how `scikit-learn` does it). 

The SVD approach does not require the computation of the covariance matrix. 
Instead, the SVD is performed directly on the data. 

In [ ]:
U, S, Vt = np.linalg.svd(scaled)

The explained variance is then the square of the singular values, scaled by the number of samples in the data (minus one, as it is a sample variance). 

In [ ]:
explained_variance = np.square(S) / (scaled.shape[0] - 1)
explained_variance

The components are then the $\mathbf{V}^\top$ columns, which are already sorted in terms of decreasing singular values. 

In [ ]:
components = Vt

transformed = components / np.sqrt(explained_variance[:, np.newaxis]) @ scaled.T
for i in range(8):
    scaled_with_pc[f'PC{i+1}'] = transformed.loc[i]
scaled_with_pc[[f'PC{i+1}' for i in range(cleaned.shape[1])]].cov()